## hyper - 01

In [1]:
import source
import numpy as np
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ae21b105 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# Importing and reshaping datatset
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

from sklearn.model_selection import train_test_split
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,test_size=0.1,stratify=y_train)

X_train = X_train.reshape(54000,-1)/255.0
X_cv = X_cv.reshape(6000,-1)/255.0
X_test = X_test.reshape(10000,-1)/255.0
y_train = source.one_hot_numpy(y_train)
y_cv = source.one_hot_numpy(y_cv)
y_test = source.one_hot_numpy(y_test)

In [3]:
arch = [784, 128, 64, 10]
activation_sequence =   ['relu', 'relu', 'softmax']
optimizer = "adam"
learning_rate = 1e-3
loss = "cross_entropy"
initialization = "Xavier"
momentum = 0.95
weight_decay = 1e-6
beta_rms = 0.95
beta_1 = 0.9
beta_2 = 0.99
md1 = source.FeedForwardNeuralNetwork(arch=arch, activation_sequence=activation_sequence, optimizer=optimizer,
                                      learning_rate=learning_rate,weight_decay= weight_decay, loss=loss,initialization=initialization,momentum=momentum,beta_rms=beta_rms,
                                      beta_1=beta_1,beta_2=beta_2)

In [4]:
batch_train = source.Batchloader(X_train, y_train, batch_size=64,shuffle=True)
batch_test = source.Batchloader(X_cv, y_cv, batch_size=64,shuffle=False)

In [5]:
epochs = 5
run = wandb.init(entity="A1_DA6401_DL", project="Reporter", config={"opt" : optimizer, "loss": loss,})
run.name = f"Hyper1_num_layers_2_act_relu_opt_{optimizer}_mnist"
for epoch in range(1,epochs+1):
    i = 0
    for X_, y_ in batch_train:
        i += 1
        md1.train_step(X_, y_, epoch)

    train_pred = md1.forward_call(inputs_=X_train)
    test_pred = md1.forward_call(inputs_=X_cv)

    loss_train = source.find_loss(y_pred= train_pred, y_true =y_train, loss="cross_entropy")
    loss_cv = source.find_loss(y_pred= test_pred, y_true = y_cv, loss = "cross_entropy")

    train_pred = md1.forward_call(inputs_=X_train,threshold=True)
    test_pred = md1.forward_call(inputs_=X_cv,threshold=True)

    accuracy_train = source.accuracy(train_pred, y_train)
    accuracy_cv = source.accuracy(test_pred,  y_cv)

    run.log(data={"loss_train" : loss_train, "loss_cv" : loss_cv, "acc_train" : accuracy_train, "acc_cv" : accuracy_cv})
    if epoch%1 == 0:
        print(F"Epoch {epoch} || loss_train : {loss_train} | loss_test : {loss_cv} || acc_train : {accuracy_train} | acc_test : {accuracy_cv} ||")

Epoch 1 || loss_train : 0.14211435861208083 | loss_test : 0.15981937495172888 || acc_train : 0.9599814814814814 | acc_test : 0.9525 ||
Epoch 2 || loss_train : 0.08919049560435206 | loss_test : 0.11473385605382856 || acc_train : 0.974462962962963 | acc_test : 0.9675 ||
Epoch 3 || loss_train : 0.06577621705005804 | loss_test : 0.09934628628464812 || acc_train : 0.9804074074074074 | acc_test : 0.9723333333333334 ||
Epoch 4 || loss_train : 0.053350674005590154 | loss_test : 0.09654993055073403 || acc_train : 0.982537037037037 | acc_test : 0.9728333333333333 ||
Epoch 5 || loss_train : 0.04415030695661414 | loss_test : 0.09610189112413399 || acc_train : 0.9853518518518518 | acc_test : 0.9735 ||


In [6]:
test_pred = md1.forward_call(inputs_=X_test,threshold=True)

y_true_test = np.argmax(y_test, axis=1)
y_pred_test = np.argmax(test_pred, axis=1)

run.log(data = {"test_acc" : source.accuracy(test_pred,y_test)})
wandb.finish()

acc_cv,▁▆███
acc_train,▁▅▇▇█
loss_cv,█▃▁▁▁
loss_train,█▄▃▂▁
test_acc,▁
acc_cv,0.9735
acc_train,0.98535
loss_cv,0.0961
loss_train,0.04415
test_acc,0.9722


## hyper - 02

In [1]:
import source
import numpy as np
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ae21b105 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# Importing and reshaping datatset
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

from sklearn.model_selection import train_test_split
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,test_size=0.1,stratify=y_train)

X_train = X_train.reshape(54000,-1)/255.0
X_cv = X_cv.reshape(6000,-1)/255.0
X_test = X_test.reshape(10000,-1)/255.0
y_train = source.one_hot_numpy(y_train)
y_cv = source.one_hot_numpy(y_cv)
y_test = source.one_hot_numpy(y_test)

In [3]:
arch = [784, 128, 128, 128, 64, 10]
activation_sequence =   ['relu', 'relu', "relu", "relu", 'softmax']
optimizer = "nadam"
learning_rate = 1e-3
loss = "cross_entropy"
initialization = "Xavier"
momentum = 0.95
weight_decay = 5e-4
beta_rms = 0.95
beta_1 = 0.9
beta_2 = 0.99
md1 = source.FeedForwardNeuralNetwork(arch=arch, activation_sequence=activation_sequence, optimizer=optimizer,
                                      learning_rate=learning_rate,weight_decay= weight_decay, loss=loss,initialization=initialization,momentum=momentum,beta_rms=beta_rms,
                                      beta_1=beta_1,beta_2=beta_2)

In [4]:
batch_train = source.Batchloader(X_train, y_train, batch_size=64,shuffle=True)
batch_test = source.Batchloader(X_cv, y_cv, batch_size=64,shuffle=False)

In [5]:
epochs = 5
run = wandb.init(entity="A1_DA6401_DL", project="Reporter", config={"opt" : optimizer, "loss": loss,})
run.name = f"Hyper2_num_layers_4_act_relu_opt_{optimizer}_weight_decay_{weight_decay}_mnist"
for epoch in range(1,epochs+1):
    i = 0
    for X_, y_ in batch_train:
        i += 1
        md1.train_step(X_, y_, epoch)

    train_pred = md1.forward_call(inputs_=X_train)
    test_pred = md1.forward_call(inputs_=X_cv)

    loss_train = source.find_loss(y_pred= train_pred, y_true =y_train, loss="cross_entropy")
    loss_cv = source.find_loss(y_pred= test_pred, y_true = y_cv, loss = "cross_entropy")

    train_pred = md1.forward_call(inputs_=X_train,threshold=True)
    test_pred = md1.forward_call(inputs_=X_cv,threshold=True)

    accuracy_train = source.accuracy(train_pred, y_train)
    accuracy_cv = source.accuracy(test_pred,  y_cv)

    run.log(data={"loss_train" : loss_train, "loss_cv" : loss_cv, "acc_train" : accuracy_train, "acc_cv" : accuracy_cv})
    if epoch%1 == 0:
        print(F"Epoch {epoch} || loss_train : {loss_train} | loss_test : {loss_cv} || acc_train : {accuracy_train} | acc_test : {accuracy_cv} ||")

Epoch 1 || loss_train : 0.11557522215831167 | loss_test : 0.12772835127945514 || acc_train : 0.9648888888888889 | acc_test : 0.9608333333333333 ||
Epoch 2 || loss_train : 0.07044924231573921 | loss_test : 0.09395070916301794 || acc_train : 0.9779814814814815 | acc_test : 0.9683333333333334 ||
Epoch 3 || loss_train : 0.06284942001922222 | loss_test : 0.0937502035837561 || acc_train : 0.9800185185185185 | acc_test : 0.9716666666666667 ||
Epoch 4 || loss_train : 0.03937305452088264 | loss_test : 0.07855537358791301 || acc_train : 0.9871481481481481 | acc_test : 0.9765 ||
Epoch 5 || loss_train : 0.03649187465209033 | loss_test : 0.08917810296969093 || acc_train : 0.9877222222222222 | acc_test : 0.9763333333333334 ||


In [6]:
test_pred = md1.forward_call(inputs_=X_test,threshold=True)

y_true_test = np.argmax(y_test, axis=1)
y_pred_test = np.argmax(test_pred, axis=1)

run.log(data = {"test_acc" : source.accuracy(test_pred,y_test)})
wandb.finish()

acc_cv,▁▄▆██
acc_train,▁▅▆██
loss_cv,█▃▃▁▃
loss_train,█▄▃▁▁
test_acc,▁
acc_cv,0.97633
acc_train,0.98772
loss_cv,0.08918
loss_train,0.03649
test_acc,0.9713


## hyper 03

In [1]:
import source
import numpy as np
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ae21b105 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# Importing and reshaping datatset
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

from sklearn.model_selection import train_test_split
X_train,X_cv,y_train,y_cv = train_test_split(X_train,y_train,test_size=0.1,stratify=y_train)

X_train = X_train.reshape(54000,-1)/255.0
X_cv = X_cv.reshape(6000,-1)/255.0
X_test = X_test.reshape(10000,-1)/255.0
y_train = source.one_hot_numpy(y_train)
y_cv = source.one_hot_numpy(y_cv)
y_test = source.one_hot_numpy(y_test)

In [3]:
arch = [784, 128, 128, 64, 10]
activation_sequence =   ['tanh', 'tanh', "tanh", 'softmax']
optimizer = "nadam"
learning_rate = 1e-3
loss = "cross_entropy"
initialization = "Xavier"
momentum = 0.95
weight_decay = 1e-4
beta_rms = 0.95
beta_1 = 0.9
beta_2 = 0.999
md1 = source.FeedForwardNeuralNetwork(arch=arch, activation_sequence=activation_sequence, optimizer=optimizer,
                                      learning_rate=learning_rate,weight_decay= weight_decay, loss=loss,initialization=initialization,momentum=momentum,beta_rms=beta_rms,
                                      beta_1=beta_1,beta_2=beta_2)

In [4]:
batch_train = source.Batchloader(X_train, y_train, batch_size=64,shuffle=True)
batch_test = source.Batchloader(X_cv, y_cv, batch_size=64,shuffle=False)

In [5]:
epochs = 6
run = wandb.init(entity="A1_DA6401_DL", project="Reporter", config={"opt" : optimizer, "loss": loss,})
run.name = f"Hyper3_num_layers_4_act_tanh_opt_{optimizer}_weight_decay_{weight_decay}_epochs_6_mnist"
for epoch in range(1,epochs+1):
    i = 0
    for X_, y_ in batch_train:
        i += 1
        md1.train_step(X_, y_, epoch)

    train_pred = md1.forward_call(inputs_=X_train)
    test_pred = md1.forward_call(inputs_=X_cv)

    loss_train = source.find_loss(y_pred= train_pred, y_true =y_train, loss="cross_entropy")
    loss_cv = source.find_loss(y_pred= test_pred, y_true = y_cv, loss = "cross_entropy")

    train_pred = md1.forward_call(inputs_=X_train,threshold=True)
    test_pred = md1.forward_call(inputs_=X_cv,threshold=True)

    accuracy_train = source.accuracy(train_pred, y_train)
    accuracy_cv = source.accuracy(test_pred,  y_cv)

    run.log(data={"loss_train" : loss_train, "loss_cv" : loss_cv, "acc_train" : accuracy_train, "acc_cv" : accuracy_cv})
    if epoch%1 == 0:
        print(F"Epoch {epoch} || loss_train : {loss_train} | loss_test : {loss_cv} || acc_train : {accuracy_train} | acc_test : {accuracy_cv} ||")

Epoch 1 || loss_train : 0.14177109551239558 | loss_test : 0.15243421994312098 || acc_train : 0.9592407407407407 | acc_test : 0.9526666666666667 ||
Epoch 2 || loss_train : 0.09155018761719622 | loss_test : 0.1078535964319808 || acc_train : 0.9736481481481482 | acc_test : 0.965 ||
Epoch 3 || loss_train : 0.06762729439221124 | loss_test : 0.09503991564784948 || acc_train : 0.9797592592592592 | acc_test : 0.969 ||
Epoch 4 || loss_train : 0.05300098112931824 | loss_test : 0.0896836260602044 || acc_train : 0.9844259259259259 | acc_test : 0.9713333333333334 ||
Epoch 5 || loss_train : 0.03872038739247794 | loss_test : 0.08166193018917839 || acc_train : 0.988462962962963 | acc_test : 0.9756666666666667 ||
Epoch 6 || loss_train : 0.03320203914838722 | loss_test : 0.08397437019490599 || acc_train : 0.9897407407407407 | acc_test : 0.9736666666666667 ||


In [6]:
test_pred = md1.forward_call(inputs_=X_test,threshold=True)

y_true_test = np.argmax(y_test, axis=1)
y_pred_test = np.argmax(test_pred, axis=1)

run.log(data = {"test_acc" : source.accuracy(test_pred,y_test)})
wandb.finish()

acc_cv,▁▅▆▇█▇
acc_train,▁▄▆▇██
loss_cv,█▄▂▂▁▁
loss_train,█▅▃▂▁▁
test_acc,▁
acc_cv,0.97367
acc_train,0.98974
loss_cv,0.08397
loss_train,0.0332
test_acc,0.9736
